In [1]:
import re
import pandas as pd

text_df = pd.read_csv("./data-2/content_description.csv", sep='\t')
text_df.head()

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...


In [2]:
corpus = []

regular_expr=r'\w+'
regular_expr_compiled = re.compile(regular_expr)

for raw_text in text_df.description.values:
    raw_text_lower = raw_text.lower()

    text_by_words = regular_expr_compiled.findall(raw_text_lower)
    corpus.append(text_by_words)
print(corpus[1])

['через', 'какие', 'трудности', 'приходится', 'проходить', 'санта', 'клаусу', 'каждый', 'год', 'чтобы', 'под', 'каждой', 'елкой', 'появился', 'тот', 'самый', 'подарок', 'расскажет', 'мульт', 'нико', '2', 'маленький', 'олененок', 'нико', 'мечтает', 'продолжить', 'дело', 'отца', 'ведь', 'его', 'папа', 'самый', 'знаменитый', 'и', 'уважаемый', 'олень', 'на', 'свете', 'о', 'его', 'работе', 'мечтает', 'каждый', 'он', 'трудится', 'в', 'упряжке', 'самого', 'санта', 'клауса', 'но', 'сам', 'нико', 'пока', 'еще', 'слишком', 'маленький', 'для', 'такого', 'тяжелого', 'и', 'ответственного', 'дела', 'так', 'что', 'пока', 'он', 'сам', 'ждет', 'санту', 'с', 'подарками', 'накануне', 'рождества', 'он', 'встречается', 'со', 'своими', 'самыми', 'близкими', 'друзьями', 'белкой', 'джулиусом', 'лаской', 'вилмой', 'и', 'сводным', 'братишкой', 'джонни', 'чтобы', 'вместе', 'встречать', 'рождество', 'и', 'не', 'пропустить', 'заветную', 'оленью', 'упряжку', 'но', 'как', 'всегда', 'в', 'пути', 'санту', 'ждут', 'нев

In [4]:
import pymorphy2

normalized_corpus = []

morph = pymorphy2.MorphAnalyzer()

for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)
print(normalized_corpus[1])

['через', 'какой', 'трудность', 'приходиться', 'проходить', 'сант', 'клаус', 'каждый', 'год', 'чтобы', 'под', 'каждый', 'ёлка', 'появиться', 'тот', 'самый', 'подарок', 'рассказать', 'мульт', 'нико', '2', 'маленький', 'оленёнок', 'нико', 'мечтать', 'продолжить', 'дело', 'отец', 'ведь', 'он', 'папа', 'самый', 'знаменитый', 'и', 'уважаемый', 'олень', 'на', 'свет', 'о', 'он', 'работа', 'мечтать', 'каждый', 'он', 'трудиться', 'в', 'упряжка', 'самый', 'сант', 'клаус', 'но', 'сам', 'нико', 'пока', 'ещё', 'слишком', 'маленький', 'для', 'такой', 'тяжёлый', 'и', 'ответственный', 'дело', 'так', 'что', 'пока', 'он', 'сам', 'ждать', 'сант', 'с', 'подарок', 'накануне', 'рождество', 'он', 'встречаться', 'с', 'свой', 'самый', 'близкие', 'друг', 'белка', 'джулиус', 'ласка', 'вилма', 'и', 'сводный', 'братишка', 'джонни', 'чтобы', 'вместе', 'встречать', 'рождество', 'и', 'не', 'пропустить', 'заветный', 'олений', 'упряжка', 'но', 'как', 'всегда', 'в', 'путь', 'сант', 'ждать', 'невероятный', 'приключение',

In [7]:
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []

for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})

tokens_df = tokens_df.assign(dummy = 1)

tokens_df.head()

,doc_id,word,dummy
0,0,хороший,1
1,0,подарок,1
2,0,который,1
3,0,только,1
4,0,можно,1


In [11]:
word_count_df = tokens_df.groupby(['doc_id', 'word'])['dummy'].count().reset_index()
word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
28,0,и,7
36,0,который,5
110,0,шерлок,4
83,0,сериал,4
76,0,с,4
8,0,весь,3
74,0,риколетти,3
104,0,холмс,2
22,0,же,2
